In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from pylab import*
import datetime as dt
import seaborn as sns


In [3]:
#Corremos el ejercicio 1 de la tarea anterior.
%run ../Tarea_1./Tarea1_ej1.ipynb

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _id_              50000 non-null  int64         
 1   _id2_             50000 non-null  int64         
 2   comp_mixto1       50000 non-null  float64       
 3   inversion         50000 non-null  float64       
 4   fecha_inicio      50000 non-null  datetime64[ns]
 5   comp_interno1     50000 non-null  float64       
 6   comp_interno2     50000 non-null  object        
 7   comp_interno3     50000 non-null  float64       
 8   comp_interno4     50000 non-null  float64       
 9   categoria1        50000 non-null  float64       
 10  categoria2        50000 non-null  float64       
 11  categoria3        50000 non-null  float64       
 12  comp_externo1     47831 non-null  float64       
 13  comp_externo2     44410 non-null  float64       
 14  comp_externo3     4999

C:\Users\Dell\AppData\Local\Temp/ipykernel_19004/2701235456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_1['porc_inv'] = final['inversion'] / tot_inv


In [4]:
#Creamos una copia de la tabla final_1 del ejercicio 1 de la tarea anterior.
copy_final_1 = final_1.copy()
copy_final_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   _id_             50000 non-null  object        
 1   mes_informacion  49352 non-null  datetime64[ns]
 2   inversion        50000 non-null  float64       
 3   porc_inv         50000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 1.9+ MB


In [5]:
#Creamos una nueva tabla sin datos faltantes en las fechas.
datos1 = copy_final_1.dropna()
datos1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   _id_             49352 non-null  object        
 1   mes_informacion  49352 non-null  datetime64[ns]
 2   inversion        49352 non-null  float64       
 3   porc_inv         49352 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 1.9+ MB


In [6]:
#Creamos una condición que nos permita filtrar a los clientes que son aptos para recibir el crédito.
mask = ((datos1['porc_inv'] > np.percentile(datos1['porc_inv'], 90)) | (mini & maxi ) )

In [7]:
#Creamos un indicador para marcar a los clientes aptos para el crédito y los que no.
datos1['target'] = 0 #No aptos.
datos1.loc[mask,'target'] = 1 #Aptos.

C:\Users\Dell\AppData\Local\Temp/ipykernel_19004/401601461.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos1['target'] = 0 #No aptos.
C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
#Las fechas las separamos de tal forma que el día, el mes y el año estén cada uno en una columna separada.
datos1['Year'] = datos1['mes_informacion'].dt.year #Año.
datos1['Month'] = datos1['mes_informacion'].dt.month #Mes.
datos1['Day'] = datos1['mes_informacion'].dt.day #Día.

C:\Users\Dell\AppData\Local\Temp/ipykernel_19004/2468330874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos1['Year'] = datos1['mes_informacion'].dt.year #Año.
C:\Users\Dell\AppData\Local\Temp/ipykernel_19004/2468330874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos1['Month'] = datos1['mes_informacion'].dt.month #Mes.
C:\Users\Dell\AppData\Local\Temp/ipykernel_19004/2468330874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [9]:
#Creamos la tabla que utilizaremos para los algoritmos que incluyen los cambios antes mencionados.
rdy_table = datos1[['_id_','Year','Month','Day','inversion','porc_inv','target']]
rdy_table.head()

,_id_,Year,Month,Day,inversion,porc_inv,target
0,78213,2021,5,1,0.000000,0.000000e+00,0
1,76457,2021,5,1,0.041039,1.258548e-08,0
2,40666,2019,5,1,0.000000,0.000000e+00,0
3,8329,2021,5,1,0.000000,0.000000e+00,0
4,68389,2021,5,1,77.422135,2.374342e-05,0
